<a href="https://colab.research.google.com/github/LucjanSakowicz/data-science-bootcamp/blob/main/07_uczenie_glebokie/02_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Tensorflow
>Strona biblioteki: [https://www.tensorflow.org/](https://www.tensorflow.org/)  
>Dokumentacja: [https://www.tensorflow.org/api_docs](https://www.tensorflow.org/api_docs)  
>
>Szybko rozwijająca się biblioteka do uczenia maszynowego i uczenia głębokiego rozwijana przez Google.
>
>Aby zainstalować bibliotekę tensorflow, użyj polecenia poniżej:
```
pip install tensorflow
```

### Keras
>Strona biblioteki: [https://keras.io/](https://keras.io/)  
>
>Wysokopoziomowy interfejs do budowy sieci neuronowych. Działa zarówno na Tensorflow, Theano oraz CNTK.
>
>Aby zainstalować bibliotekę keras, użyj polecenia poniżej:
```
pip install keras
```

### Model sekwencjny (Sequential Model):
1. [Wprowadzenie - pierwszy model](#a0)
2. [Funkcje aktywacji](#a1)
3. [Kompilacja modelu](#a2)
4. [Trenowanie modelu](#a3)
5. [Przykład - klasyfikacja binarna](#a4)
6. [Przykład - klasyfikacja wieloklasowa](#a5)
7. [Przykład - regresja](#a6)



Załadowanie biblioteki tensorflow


In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.12.0'

### <a name='a0'></a>Wprowadzenie - pierwszy model
Model sekwencyjny to nic innego jak liniowy stos warstw.

In [2]:
# utworzenie instancji klasy Sequential
from tensorflow.keras.models import Sequential

model = Sequential()
print(model)

Podstawowym elementem składowym modelu są warstwy. Aby dodać najbardziej standardową warstwę - warstwę gęsto połączoną należy użyć warstwy **Dense**. Aby dodać warstwę do modelu należy użyć metody *.add()*

In [3]:
from tensorflow.keras.layers import Dense

model.add(Dense(units=4, input_shape=(10,)))

Wyświetlenie podsumowania modelu: metoda *.summary()*

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


Dodanie kolejnej warstwy

In [5]:
model.add(Dense(units=2))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 44        
                                                                 
 dense_1 (Dense)             (None, 2)                 10        
                                                                 
Total params: 54
Trainable params: 54
Non-trainable params: 0
_________________________________________________________________


### <a name='a1'></a>Funkcje aktywacji
Istotnym elemenem sieci neuronowych jest dobór odpowiednich funkcji aktywacji. Funkcje aktywacji jak sama nazwa wskazuje są odpowiedzialne za aktywowanie odpowiednich neuronów podczas procesu uczenia.

Jeżeli nie określimy podczas dodawania warstwy funkcji aktywacji, domyślnie stosowana jest liniowa funkcja aktywacji, tzn. $a(x)=x$

Warstwa z liniową funkcją aktywacji może uczyć się tylko liniowych przekształceń danych wejściowych. Dlatego stosuje się różne funkcje aktywacji aby rozwiazywać problemy nieliniowe.

[Keras: Funkcje aktywacji](https://keras.io/activations/)

In [6]:
from tensorflow.keras.activations import linear

random_data = np.linspace(start=-3, stop=3, num=300)
data = pd.DataFrame({'data': random_data, 'linear': linear(random_data)})
data.head()

,data,linear
0,-3.000000,-3.000000
1,-2.979933,-2.979933
2,-2.959866,-2.959866
3,-2.939799,-2.939799
4,-2.919732,-2.919732


In [7]:
px.line(data, x='data', y='linear', width=500, height=400, range_y=[-3, 3])

In [8]:
from tensorflow.keras.activations import sigmoid

data = pd.DataFrame({'data': random_data, 'sigmoid': sigmoid(random_data)})
data.head()

,data,sigmoid
0,-3.000000,0.047426
1,-2.979933,0.048341
2,-2.959866,0.049272
3,-2.939799,0.050221
4,-2.919732,0.051187


In [9]:
px.line(data, x='data', y='sigmoid', width=500, height=400, range_y=[-0.5, 1.5])

In [10]:
from tensorflow.keras.activations import relu

data = pd.DataFrame({'data': random_data, 'relu': relu(random_data)})
data.head()

,data,relu
0,-3.000000,0.0
1,-2.979933,0.0
2,-2.959866,0.0
3,-2.939799,0.0
4,-2.919732,0.0


In [11]:
px.line(data, x='data', y='relu', width=500, height=400, range_y=[-0.5, 1.5])

In [12]:
from tensorflow.keras.activations import tanh

data = pd.DataFrame({'data': random_data, 'tanh': tanh(random_data)})
data.head()

,data,tanh
0,-3.000000,-0.995055
1,-2.979933,-0.994853
2,-2.959866,-0.994643
3,-2.939799,-0.994424
4,-2.919732,-0.994196


In [13]:
px.line(data, x='data', y='tanh', width=500, height=400, range_y=[-1.5, 1.5])

In [14]:
model = Sequential()
model.add(Dense(units=8, activation='relu', input_shape=(10,)))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 88        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### <a name='a2'></a>Kompilacja modelu
Przed rozpoczęciem trenowania sieci należy odpowiednio skonfigurować proces uczenia. W tym kroku określamy:
* rodzaj optymalizatora ([Keras - Optymalizatory](https://keras.io/optimizers/)) 
* funkcję straty ([Keras - Funkcje Straty](https://keras.io/losses/))
* metryki, które będziemy obserwować podczas trenowania sieci ([Keras - Metryki](https://keras.io/metrics/))

In [15]:
# klasyfikacja binarna
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# klasyfikacja wieloklasowa
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# regresja
model.compile(optimizer='rmsprop',
              loss='mse')

### <a name='a3'></a>Trenowanie modelu
Za dane wejściowe do modelu należy przekazać Numpy arrays:
* **epochs** - krotność przejścia danych przez sieć w procesie uczenia
* **batch_size** - rozmiar wsadu po którym następuje aktualizacja wag
* **validation_split** - część danych treningowych, które zostaną wykorzystane jako zbiór walidacyjny
* **validation_data** - (x_val, y_val) - dane wykorzystane do walidacji modelu

In [ ]:
# model.fit(data, labels, epochs=10, batch_size=32)
# model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)
# model.fit(data, labels, epochs=10, batch_size=32, validation_data=(x_val, y_val))

### <a name='a4'></a> Przykład - klasyfikacja binarna

In [16]:
data = np.random.randn(10000, 150)
labels = np.random.randint(2, size=(10000, 1))

print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [17]:
data[:3]

array([[ 1.57967076,  1.6185848 , -1.58438842,  0.6692536 ,  0.78407367,
         0.21433414,  0.24012015, -0.48740191, -0.66621075, -0.34907257,
         0.41386606, -0.46716295,  0.39689882,  0.33803277,  0.82083134,
        -0.45665805, -1.0799634 , -0.81413655, -0.23400966, -0.52975988,
         0.08577751,  0.36714354, -1.45946318, -1.6109715 ,  0.97619928,
        -0.97606914, -0.62119552,  0.83946467,  0.33765314, -0.79108554,
         3.06081458, -0.32543904,  0.26654318, -0.56623724, -1.3346221 ,
        -1.62218583,  1.035004  ,  0.16154006,  0.18119017, -0.12189947,
        -1.60872581, -0.37355537, -0.74345477,  0.04743039, -0.43674745,
         0.55849259,  1.495437  ,  0.57244358,  2.07288601,  1.35163492,
        -0.60303969,  0.6209877 , -0.62237189,  0.2522064 ,  0.51292598,
         1.07283146, -1.17698158,  0.77775501,  0.44080583,  0.75750729,
        -0.32781629,  0.91381652, -0.39311991, -0.69165233, -2.42882103,
         0.71358077,  0.68206571,  0.3560151 , -0.5

In [18]:
labels[:10]

array([[1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [19]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20)

Epoch 1/20
313/313 [==============================] - 1s 2ms/step - loss: 0.7579 - accuracy: 0.4901
Epoch 2/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6976 - accuracy: 0.5284
Epoch 3/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6813 - accuracy: 0.5603
Epoch 4/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6685 - accuracy: 0.5888
Epoch 5/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6562 - accuracy: 0.6095
Epoch 6/20
313/313 [==============================] - 1s 3ms/step - loss: 0.6444 - accuracy: 0.6317
Epoch 7/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6304 - accuracy: 0.6468
Epoch 8/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6163 - accuracy: 0.6701
Epoch 9/20
313/313 [==============================] - 1s 2ms/step - loss: 0.6019 - accuracy: 0.6829
Epoch 10/20
313/313 [==============================] - 1s 2ms/step - loss: 0.5886 - accuracy: 0.6923

In [20]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=64)

Epoch 1/20
157/157 [==============================] - 1s 2ms/step - loss: 0.7553 - accuracy: 0.5003
Epoch 2/20
157/157 [==============================] - 0s 2ms/step - loss: 0.7001 - accuracy: 0.5314
Epoch 3/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.5622
Epoch 4/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6716 - accuracy: 0.5863
Epoch 5/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6608 - accuracy: 0.6024
Epoch 6/20
157/157 [==============================] - 0s 3ms/step - loss: 0.6503 - accuracy: 0.6230
Epoch 7/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6398 - accuracy: 0.6362
Epoch 8/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6291 - accuracy: 0.6524
Epoch 9/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6179 - accuracy: 0.6676
Epoch 10/20
157/157 [==============================] - 0s 2ms/step - loss: 0.6071 - accuracy: 0.6799

In [21]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
250/250 [==============================] - 1s 4ms/step - loss: 0.7512 - accuracy: 0.5015 - val_loss: 0.7250 - val_accuracy: 0.4925
Epoch 2/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6966 - accuracy: 0.5454 - val_loss: 0.7231 - val_accuracy: 0.4855
Epoch 3/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6768 - accuracy: 0.5810 - val_loss: 0.7218 - val_accuracy: 0.4980
Epoch 4/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6618 - accuracy: 0.6040 - val_loss: 0.7238 - val_accuracy: 0.4890
Epoch 5/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6470 - accuracy: 0.6340 - val_loss: 0.7290 - val_accuracy: 0.4855
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6331 - accuracy: 0.6472 - val_loss: 0.7347 - val_accuracy: 0.4910
Epoch 7/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6181 - accuracy: 0.6693 - val_loss: 0.7444 - val_accuracy: 0.4845
Epoch 

In [22]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

In [23]:
metrics = history.history
metrics.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [24]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/20
250/250 [==============================] - 1s 2ms/step - loss: 0.7644 - accuracy: 0.5021 - val_loss: 0.7298 - val_accuracy: 0.5080
Epoch 2/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6983 - accuracy: 0.5424 - val_loss: 0.7264 - val_accuracy: 0.4995
Epoch 3/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6769 - accuracy: 0.5770 - val_loss: 0.7273 - val_accuracy: 0.4980
Epoch 4/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6612 - accuracy: 0.6015 - val_loss: 0.7301 - val_accuracy: 0.4970
Epoch 5/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6465 - accuracy: 0.6251 - val_loss: 0.7294 - val_accuracy: 0.5055
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.6316 - accuracy: 0.6488 - val_loss: 0.7354 - val_accuracy: 0.5055
Epoch 7/20
250/250 [==============================] - 0s 2ms/step - loss: 0.6162 - accuracy: 0.6649 - val_loss: 0.7394 - val_accuracy: 0.5060
Epoch 

In [25]:
test_data = np.random.randn(5, 150)
test_labels = np.random.randint(2, size=(5, 1))

In [26]:
model.predict(test_data)

1/1 [==============================] - 0s 78ms/step


array([[0.53481174],
       [0.07621659],
       [0.72241604],
       [0.837024  ],
       [0.8240422 ]], dtype=float32)

In [28]:
#model.predict_classes(test_data) nie ma tej metody w tensorflow 2.x

#zamiast tego

np.argmax(model.predict(test_data), axis=1)

1/1 [==============================] - 0s 25ms/step


array([0, 0, 0, 0, 0])

### <a name='a5'></a> Przykład - klasyfikacja wieloklasowa

In [29]:
data = np.random.random((10000, 150))
labels = np.random.randint(10, size=(10000, 1))

In [30]:
print(data.shape)
print(labels.shape)

(10000, 150)
(10000, 1)


In [31]:
labels[:10]

array([[8],
       [5],
       [8],
       [4],
       [4],
       [6],
       [2],
       [4],
       [9],
       [2]])

In [32]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=10) #zmieni klasy 0-9 na zapis kategoryczny
labels

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
labels[1]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [34]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=30, validation_split=0.2)

Epoch 1/30
250/250 [==============================] - 1s 3ms/step - loss: 2.3305 - accuracy: 0.1055 - val_loss: 2.3109 - val_accuracy: 0.1070
Epoch 2/30
250/250 [==============================] - 1s 3ms/step - loss: 2.3026 - accuracy: 0.1093 - val_loss: 2.3081 - val_accuracy: 0.0965
Epoch 3/30
250/250 [==============================] - 1s 3ms/step - loss: 2.3001 - accuracy: 0.1090 - val_loss: 2.3061 - val_accuracy: 0.1005
Epoch 4/30
250/250 [==============================] - 1s 2ms/step - loss: 2.2979 - accuracy: 0.1114 - val_loss: 2.3105 - val_accuracy: 0.0945
Epoch 5/30
250/250 [==============================] - 1s 3ms/step - loss: 2.2947 - accuracy: 0.1209 - val_loss: 2.3085 - val_accuracy: 0.1025
Epoch 6/30
250/250 [==============================] - 1s 2ms/step - loss: 2.2914 - accuracy: 0.1184 - val_loss: 2.3125 - val_accuracy: 0.0990
Epoch 7/30
250/250 [==============================] - 0s 2ms/step - loss: 2.2877 - accuracy: 0.1293 - val_loss: 2.3115 - val_accuracy: 0.0925
Epoch 

In [35]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 69ms/step


array([[0.09220319, 0.12801883, 0.20586197, 0.08867673, 0.09478191,
        0.1078842 , 0.10667214, 0.04698197, 0.07692315, 0.05199589],
       [0.09826679, 0.1817261 , 0.07102354, 0.23364238, 0.07549641,
        0.14352816, 0.03252519, 0.02823509, 0.053277  , 0.0822793 ],
       [0.11396034, 0.07708387, 0.1563843 , 0.15180339, 0.0849565 ,
        0.0943841 , 0.11228225, 0.09110877, 0.05166654, 0.06636994],
       [0.0801392 , 0.07241483, 0.04285774, 0.38605675, 0.0856216 ,
        0.03210002, 0.08474153, 0.04934748, 0.09955774, 0.0671631 ],
       [0.06533127, 0.04964114, 0.09165062, 0.08867949, 0.08391672,
        0.09576909, 0.18873398, 0.16986565, 0.09320401, 0.07320797],
       [0.15887208, 0.04078841, 0.17133401, 0.18022029, 0.05038819,
        0.06680838, 0.07351971, 0.07361671, 0.06737577, 0.11707643],
       [0.06824606, 0.03518648, 0.12659474, 0.06767865, 0.04749248,
        0.19769289, 0.15405384, 0.09682529, 0.09097036, 0.11525913],
       [0.0850554 , 0.11792107, 0.1076125

In [36]:
np.argmax(model.predict(test_data), axis=1)

1/1 [==============================] - 0s 20ms/step


array([2, 3, 2, 3, 6, 3, 5, 5, 8, 9])

### <a name='a6'></a> Przykład - regresja

In [37]:
data = np.random.random((10000, 150))
labels = 50 * np.random.random(10000)

In [38]:
labels[:10]

array([40.59083391, 30.56900679, 45.41174098, 42.84481717,  6.88500264,
        8.77202493, 21.4100912 ,  0.88530445, 24.94135983, 18.58288645])

In [39]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mse')

model.fit(data, labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
250/250 [==============================] - 1s 3ms/step - loss: 261.3337 - val_loss: 210.1762
Epoch 2/30
250/250 [==============================] - 1s 2ms/step - loss: 214.4634 - val_loss: 210.1111
Epoch 3/30
250/250 [==============================] - 0s 2ms/step - loss: 213.9314 - val_loss: 212.5753
Epoch 4/30
250/250 [==============================] - 1s 2ms/step - loss: 213.9583 - val_loss: 210.0416
Epoch 5/30
250/250 [==============================] - 1s 2ms/step - loss: 213.6501 - val_loss: 211.9289
Epoch 6/30
250/250 [==============================] - 1s 2ms/step - loss: 213.3020 - val_loss: 210.0339
Epoch 7/30
250/250 [==============================] - 0s 2ms/step - loss: 213.1105 - val_loss: 210.3126
Epoch 8/30
250/250 [==============================] - 1s 2ms/step - loss: 212.5989 - val_loss: 212.2951
Epoch 9/30
250/250 [==============================] - 0s 2ms/step - loss: 212.6945 - val_loss: 210.9590
Epoch 10/30
250/250 [==============================] - 1s 2ms/st

In [40]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(150,)))
model.add(Dense(units=1))

model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mse'])

model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
250/250 [==============================] - 1s 3ms/step - loss: 14.0284 - mse: 283.7479 - val_loss: 12.5540 - val_mse: 210.8035
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 12.6951 - mse: 215.2140 - val_loss: 12.5286 - val_mse: 209.9423
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 12.6865 - mse: 214.8904 - val_loss: 12.5146 - val_mse: 209.7056
Epoch 4/10
250/250 [==============================] - 1s 3ms/step - loss: 12.6637 - mse: 214.6040 - val_loss: 12.5433 - val_mse: 210.4011
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 12.6528 - mse: 214.2749 - val_loss: 12.5396 - val_mse: 211.6432
Epoch 6/10
250/250 [==============================] - 1s 2ms/step - loss: 12.6390 - mse: 213.9116 - val_loss: 12.5922 - val_mse: 212.2940
Epoch 7/10
250/250 [==============================] - 1s 2ms/step - loss: 12.6370 - mse: 213.6621 - val_loss: 12.6045 - val_mse: 212.7684
Epoch 8/10
250/250 [==============

In [41]:
test_data = np.random.random((10, 150))

model.predict(test_data)

1/1 [==============================] - 0s 52ms/step


array([[23.474909],
       [23.559319],
       [22.979675],
       [26.253942],
       [24.075138],
       [24.930748],
       [25.282522],
       [25.266874],
       [23.19204 ],
       [24.858475]], dtype=float32)